In [2]:
import ast

In [4]:
a = "name = 'iPhone'"
a

"name = 'iPhone'"

In [9]:
a_tree = ast.parse(a)
print(ast.dump(a_tree, indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='name', ctx=Store())],
      value=Constant(value='iPhone'))],
  type_ignores=[])


In [10]:
addfunc = '''
a = 1024
b = 2032
c = a + b
print(c)
'''
add_tree = ast.parse(addfunc)
print(ast.dump(add_tree, indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='a', ctx=Store())],
      value=Constant(value=1024)),
    Assign(
      targets=[
        Name(id='b', ctx=Store())],
      value=Constant(value=2032)),
    Assign(
      targets=[
        Name(id='c', ctx=Store())],
      value=BinOp(
        left=Name(id='a', ctx=Load()),
        op=Add(),
        right=Name(id='b', ctx=Load()))),
    Expr(
      value=Call(
        func=Name(id='print', ctx=Load()),
        args=[
          Name(id='c', ctx=Load())],
        keywords=[]))],
  type_ignores=[])


In [11]:
add_code = compile(add_tree, 'add_tree', 'exec')
exec(add_code)

3056


In [18]:
import pprint
print(ast.dump(add_tree,indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='a', ctx=Store())],
      value=Constant(value=1024)),
    Assign(
      targets=[
        Name(id='b', ctx=Store())],
      value=Constant(value=2032)),
    Assign(
      targets=[
        Name(id='c', ctx=Store())],
      value=BinOp(
        left=Name(id='a', ctx=Load()),
        op=Add(),
        right=Name(id='b', ctx=Load()))),
    Expr(
      value=Call(
        func=Name(id='print', ctx=Load()),
        args=[
          Name(id='c', ctx=Load())],
        keywords=[]))],
  type_ignores=[])


In [20]:
add_tree.body[2].value.op # this is the add
add_tree.body[2].value.op = ast.Mult()
print(ast.dump(add_tree, indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='a', ctx=Store())],
      value=Constant(value=1024)),
    Assign(
      targets=[
        Name(id='b', ctx=Store())],
      value=Constant(value=2032)),
    Assign(
      targets=[
        Name(id='c', ctx=Store())],
      value=BinOp(
        left=Name(id='a', ctx=Load()),
        op=Mult(),
        right=Name(id='b', ctx=Load()))),
    Expr(
      value=Call(
        func=Name(id='print', ctx=Load()),
        args=[
          Name(id='c', ctx=Load())],
        keywords=[]))],
  type_ignores=[])


In [21]:
add_code = compile(add_tree, 'add_tree', 'exec')
exec(add_code) # it multiplied!

2080768


In [25]:
class_code = """ 
class MyClass:  
    class_id = 1001
    @staticmethod
    def get_something(self, ):
        return 'thing'
"""

class_tree = ast.parse(class_code)
print(ast.dump(class_tree, indent=2))

Module(
  body=[
    ClassDef(
      name='MyClass',
      bases=[],
      keywords=[],
      body=[
        Assign(
          targets=[
            Name(id='class_id', ctx=Store())],
          value=Constant(value=1001)),
        FunctionDef(
          name='get_something',
          args=arguments(
            posonlyargs=[],
            args=[
              arg(arg='self')],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]),
          body=[
            Return(
              value=Constant(value='thing'))],
          decorator_list=[
            Name(id='staticmethod', ctx=Load())])],
      decorator_list=[])],
  type_ignores=[])


In [26]:
instance_code = """instance = MyClass()"""
instance_tree = ast.parse(instance_code)
print(ast.dump(instance_tree, indent=4))

Module(
    body=[
        Assign(
            targets=[
                Name(id='instance', ctx=Store())],
            value=Call(
                func=Name(id='MyClass', ctx=Load()),
                args=[],
                keywords=[]))],
    type_ignores=[])


In [27]:
print(ast.unparse(instance_tree))

instance = MyClass()


In [28]:
from ast import NodeTransformer

In [29]:
class ModifyVegCounter(NodeTransformer):
    def visit_Constant(self, node):


        modifiedValue = ast.Constant('item:' + str(node.
                                           value))
        return modifiedValue


In [55]:
import sys
from ast import parse, dump

def parseprint(code, filename="<string>", mode="exec", type_comments=False,
               **kwargs):
    """Parse some code from a string and pretty-print it."""
    if sys.version_info >= (3, 8):
        node = parse(code, mode=mode, type_comments=type_comments)
    else:
        node = parse(code, mode=mode)   # An ode to the code
    print(dump(node, **kwargs, indent=2))


In [56]:
parseprint("func(a, b=c, *d, **e)")


Module(
  body=[
    Expr(
      value=Call(
        func=Name(id='func', ctx=Load()),
        args=[
          Name(id='a', ctx=Load()),
          Starred(
            value=Name(id='d', ctx=Load()),
            ctx=Load())],
        keywords=[
          keyword(
            arg='b',
            value=Name(id='c', ctx=Load())),
          keyword(
            value=Name(id='e', ctx=Load()))]))],
  type_ignores=[])


# With Pandas

In [32]:
from pandas import DataFrame, Series

df = DataFrame({
    'id': ['a', 'b'],
    'list_column': ['[1, 2, 3]', '[4, 5, 6]']
})
df.head()

,id,list_column
0,a,"[1, 2, 3]"
1,b,"[4, 5, 6]"


In [33]:
df['list_column'].apply(ast.literal_eval)

0    [1, 2, 3]
1    [4, 5, 6]
Name: list_column, dtype: object

# Visiting Nodes
 
From greentreesnakes:   
`ast.NodeVisitor` is the primary tool for ‘scanning’ the tree. To use it, subclass it and override methods visit_Foo, corresponding to the node classes (see Meet the Nodes).

For example, this visitor will print the names of any functions defined in the given code, including methods and functions defined within other functions:

In [50]:
class FuncLister(ast.NodeVisitor):
    def visit_FunctionDef(self, node):
        print(node.name)
        self.generic_visit(node)

FuncLister().visit(class_tree) # OH


get_something
